In [ ]:
%cd compositional-translation

In [ ]:
from comptra.sampler import cohereSampler

In [3]:
from comptra.retriever import Retriever

Unable to initialize backend 'cuda': 
Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [4]:
retriever = Retriever(
    dataset_name_or_path="flores",
    retriever_type="bm25s",
    source_language="English",
    target_language="French"
)

Building index from IDs objects                       


In [5]:
sampler = cohereSampler(
    model_name_or_path="command-r-08-2024",
    tokenizer_name_or_path=None,
    src="English",
    tgt="French",
    template=None,
    merge_prompt="vanilla",
    # method_translate="vanilla",
    method_translate="nllb",
    selection_method="comet-qe",
    nllb_name_or_path="facebook/nllb-200-distilled-600M",
    method_divide=None
)

We use an instruction fine-tuned model!
We are going to use NLLB, precisely facebook/nllb-200-distilled-600M
Merge algorithm: vanilla, selection method: comet-qe


In [ ]:
dico = {}
sentence = "I am very inclined towards eating your pancreas to alleviate my hunger."
t = sampler.translate(
    sentences=[sentence], temperature=0.0, num_return_sequences=1, do_sample=False
)
print(f"NLLB: {t[0]}")
dico["NLLB"] = t[0]
#"""
outputs = sampler.generate(
    prompts=[
        f"What is the French translation of the following sentence\n\n{sentence}"
    ],
    max_new_tokens=100,
    top_p=1.0,
    temperature=0,
)
print(f"\n---zero-shot MT\n---: {outputs[0]}")
outputs = sampler.refine([sentence], [outputs[0]], 1)
dico["ZS"] = outputs[0]
print(f"\n---zero-shot MT + Refine\n---: {outputs[0]}")
dico["ZSREFINE"] = outputs[0]
outputs = sampler.cot(
    prompts=[
        f"What is the French translation of the following sentence\n\n{sentence}\n\nLet's think step by step."
    ],
    sentences=[sentence],
    max_new_tokens=500,
)
print(f"zero-shot CoT MT: {outputs[0]}")
dico["ZSCoT"] = outputs[0]

outputs = sampler.maps(sentences=[sentence], max_new_tokens=500)
print(f"MAPS MT: {outputs[0]}")
dico["MAPS"] = outputs[0]

outputs = sampler.step_by_step(sentences=[sentence], max_new_tokens=500)
print(f"SBYS: {outputs[0]}")
dico["SBYS"] = outputs[0]

demonstrations = retriever.query(
    sentence, 5
)

outputs = sampler.tear(sentences=[sentence], demonstrations=[demonstrations], max_new_tokens=500)
print(f"TEaR: {outputs[0]}")
dico["TEaR"] = outputs[0]
# CompTra
sentences = [sentence]
subsentences = sampler.divide(n_splits=-1, sentences=sentences)
print(subsentences)
translations = []
for i in range(len(subsentences)):
    t = sampler.translate(
        sentences=subsentences[i],
        demonstrations=[[] for _ in range(len(subsentences[i]))],
    )
    translations.append(t)
print(translations)
comptra_translations = sampler.merge(
    sentences=sentences,
    inputs=subsentences,
    outputs=translations,
    max_new_tokens=300,
)
print(f"CompTra translation: {comptra_translations[0]}")
dico["CompTra"] = comptra_translations[0]
refine_translations = sampler.refine(
    sentences = sentences,
    prev_translations = comptra_translations,
    number_of_refining_steps = 1
)
dico["CompTra + Refine"] = refine_translations[0]

In [12]:
for key in dico:
    print(f"{key}: {dico[key]}")

NLLB: Je suis très enclin à manger votre pancréas pour soulager ma faim.
ZS: "Je suis fortement tenté de manger votre pancréas pour apaiser ma faim."
ZSREFINE: "Je suis fortement tenté de manger votre pancréas pour apaiser ma faim."
ZSCoT: Je suis très enclin à manger votre pancréas pour soulager ma faim.
MAPS: Je suis très enclin à manger ton pancréas pour apaiser ma faim.
SBYS: "Je suis fortement tenté de manger votre pancréas pour apaiser ma faim."
TEaR: Je suis très tenté de manger votre pancréas pour apaiser ma faim.
CompTra: Je suis très enclin à manger votre pancréas pour apaiser ma faim.
CompTra + Refine: Je suis fortement tenté de manger votre pancréas pour apaiser ma faim.
